In [1]:
# To Do
# * redo data pre processing with the new t_y in mind [X]
# * Figure out why we ahve that array missmatch in Ys (you have to pad it with 121 0s at the end) []
#   * my assumption is that it's because when stride is done on t_x it just produces different t_y then in the original notebook (t_x was different in the original notebook)
# * Make sure you have your model evaluation tools/metrics set up (ie f1 score over time) before you train your model on big datasets



In [2]:
# model architecture

# Input

# Conv1D
# BatchNorm
# Activation
# Dropout

# GRU
# Dropout
# BatchNorm

# GRU
# Dropout
# BatchNorm
# Dropout

# Dense
# TimeDistributed

In [3]:
import h5py

In [6]:
hdf5_f = r"test.h5"
dataset_name = "X_train.npy"

file = h5py.File(hdf5_f, 'r+')
dataset = file[dataset_name]

KeyError: "Unable to open object (object 'X_train.npy' doesn't exist)"

In [3]:
import numpy as np

In [4]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [ ]:
X_train = np.load("X_train.npy")

In [ ]:
# load data (data was preprocessed with data_prep notebook and uploaded to gdrive)
X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")

In [6]:
X_train.shape, Y_train.shape

((353, 5998, 101), (353, 1496))

In [7]:
#Y_train = np.pad(Y_train, ((0,0),(0,121)), 'constant')

In [8]:
#tmp.shape

In [9]:
# evaluate the model with ROC 

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [10]:
t_x = 5998
number_of_frequencies = 101

In [11]:
input_x = Input(shape=(t_x, number_of_frequencies))

X = Conv1D(filters=196, kernel_size=15, strides=4)(input_x)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Dropout(rate=0.8)(X)

X = GRU(units=128, return_sequences=True)(X)
X = Dropout(rate=0.8)(X)
X = BatchNormalization()(X)

X = GRU(units=128, return_sequences=True)(X)
X = Dropout(rate=0.8)(X)
X = BatchNormalization()(X)
X = Dropout(rate=0.8)(X)

X = TimeDistributed(Dense(1, activation="sigmoid"))(X)

model = Model(inputs=input_x, outputs=X)

In [12]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)

In [13]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy", f1_m])

In [14]:
X_train.shape

(353, 5998, 101)

In [15]:
Y_train.shape

(353, 1496)

In [18]:
Y_train = Y_train.reshape((353, 1496, 1))

In [19]:
Y_train.shape

(353, 1496, 1)

In [20]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5998, 101)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1496, 196)         297136    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1496, 196)         784       
_________________________________________________________________
activation_1 (Activation)    (None, 1496, 196)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1496, 196)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 1496, 128)         124800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1496, 128)         0   

In [ ]:
model.fit(X_train, Y_train, batch_size=6, epochs=12)

Epoch 1/12
353/353 [==============================] - 650s 2s/step - loss: 1.2976 - accuracy: 0.5082 - f1_m: 0.1282
Epoch 2/12
353/353 [==============================] - 648s 2s/step - loss: 1.1549 - accuracy: 0.5239 - f1_m: 0.1273
Epoch 3/12
353/353 [==============================] - 648s 2s/step - loss: 1.0560 - accuracy: 0.5521 - f1_m: 0.1264
Epoch 4/12
353/353 [==============================] - 638s 2s/step - loss: 1.0228 - accuracy: 0.5690 - f1_m: 0.1246
Epoch 5/12
353/353 [==============================] - 636s 2s/step - loss: 1.0017 - accuracy: 0.5899 - f1_m: 0.1216
Epoch 6/12
353/353 [==============================] - 637s 2s/step - loss: 0.9860 - accuracy: 0.6046 - f1_m: 0.1189
Epoch 7/12
353/353 [==============================] - 639s 2s/step - loss: 0.9699 - accuracy: 0.6186 - f1_m: 0.1168
Epoch 8/12
353/353 [==============================] - 650s 2s/step - loss: 0.9603 - accuracy: 0.6322 - f1_m: 0.1139
Epoch 9/12
353/353 [==============================] - 642s 2s/step - los